In [1]:
from PIL import Image
import time
import numpy as np

s = time.time()

for i in range(1):
    img = Image.open('/media/kyle/store/insta_data/hashtags/#buzzfeast/2019-02-26_23-00-09_UTC.jpg')
    img = img.resize((256,256))

e = time.time()
print(np.asarray(img).transpose(2,0,1).shape)

print(e-s)

FileNotFoundError: [Errno 2] No such file or directory: '/media/kyle/store/insta_data/hashtags/#buzzfeast/2019-02-26_23-00-09_UTC.jpg'

In [17]:
import os,sys,cv2,gc,json,lzma,re
import numpy as np
import torch
import pickle as pkl
import torch
from imageio import imread
from PIL import Image
import glob

xdim = 256
ydim = 256
metadata = []

dfolder = '/home/jok120/ml/proj/data/old_data/tags/tmp/'

for folder in os.listdir(dfolder):
    n = 0
    n_int = 1

    gc.collect()
    
    path_folder = os.path.join(dfolder,folder)

    if not (os.path.isdir(path_folder)):# and folder.startswith('#')):
        continue
        
    jpgCounter = len(glob.glob1(path_folder,"*.jpg"))
    metadataCounter = len(glob.glob1(path_folder,"*.json.xz"))
    print(jpgCounter, "JPGs,", metadataCounter, "metas in {}.".format(folder))
    images = torch.empty(jpgCounter,3,xdim,ydim)
    stored_indices = []

    for f in os.listdir(path_folder):
        fname, fext = os.path.splitext(f)

        if fext != '.jpg':
            continue

        #img = imread(os.path.join(dfolder,folder,f)) / 255
        #img = torch.from_numpy(cv2.resize(img, dsize=(ydim,xdim)).transpose(2,0,1)[None,:]).type(torch.FloatTensor)
        img = Image.open(os.path.join(dfolder,folder,f))
        img = img.resize((ydim,xdim))
        img = np.asarray(img) / 255
        img = torch.from_numpy(img.transpose(2,0,1)).type(torch.FloatTensor)

        json_filename = os.path.join(dfolder,folder,re.sub('UTC_(\\d+)','UTC',fname)+'.json.xz')

        try:
            json_str = lzma.open(json_filename).read()
        except:
            continue
        json_data = json.loads(json_str)
        metadata.append(json_data)
        
        images[n] = img
        stored_indices.append(n)
        n += 1
        

        if n % 100 == 0:
            print('{} - {} images processed'.format(folder,n),end='\r')
    images = images[stored_indices]
    print('{} - finished processing - output shape - {}'.format(folder,images.shape))
    assert images.shape[0] == len(metadata), "non-matching number of images/metas"
    torch.save(images,os.path.join(dfolder,'{}.tch'.format(folder)))
    pkl.dump(metadata,open(os.path.join(dfolder,'{}_metadata.pkl'.format(folder)),'wb'))
    
    images = torch.empty(0,3,xdim,ydim)
    metadata = []

4779 JPGs, 4271 metas in lifeandthyme.
lifeandthyme - finished processing - output shape - torch.Size([4778, 3, 256, 256])
388 JPGs, 380 metas in kimberleyhasselbrink.
kimberleyhasselbrink - finished processing - output shape - torch.Size([387, 3, 256, 256])
1500 JPGs, 1494 metas in julieskitchen.
julieskitchen - finished processing - output shape - torch.Size([1499, 3, 256, 256])
570 JPGs, 563 metas in hungrybetches.
hungrybetches - finished processing - output shape - torch.Size([569, 3, 256, 256])
1853 JPGs, 1795 metas in food_feels.
food_feels - finished processing - output shape - torch.Size([1852, 3, 256, 256])
1 JPGs, 1 metas in local_milk.
local_milk - finished processing - output shape - torch.Size([0, 3, 256, 256])
946 JPGs, 917 metas in izyhossack.
izyhossack - finished processing - output shape - torch.Size([945, 3, 256, 256])
1409 JPGs, 1387 metas in candidsbyjo.
candidsbyjo - finished processing - output shape - torch.Size([1408, 3, 256, 256])
304 JPGs, 303 metas in girle

jakesfoodbible - finished processing - output shape - torch.Size([178, 3, 256, 256])
1350 JPGs, 1316 metas in cashewkitchen.
cashewkitchen - finished processing - output shape - torch.Size([1349, 3, 256, 256])
2813 JPGs, 2582 metas in evansungnyc.
evansungnyc - finished processing - output shape - torch.Size([2807, 3, 256, 256])
837 JPGs, 703 metas in cocktailsbykurtis.
cocktailsbykurtis - finished processing - output shape - torch.Size([833, 3, 256, 256])
427 JPGs, 416 metas in thewholefooddiary.
thewholefooddiary - finished processing - output shape - torch.Size([426, 3, 256, 256])
1367 JPGs, 1352 metas in em.peachy.
em.peachy - finished processing - output shape - torch.Size([1366, 3, 256, 256])
963 JPGs, 918 metas in maurizio.
maurizio - finished processing - output shape - torch.Size([962, 3, 256, 256])
3514 JPGs, 3464 metas in jakecohen.
jakecohen - finished processing - output shape - torch.Size([3503, 3, 256, 256])
341 JPGs, 305 metas in peacelovedonutsnaplesfl.
peacelovedonuts

ValueError: axes don't match array

In [14]:
m = pickle.load(open(os.path.join('/home/jok120/ml/proj/data/old_data/tags/tmp/abrowntable_metadata.pkl'),'rb'))
d = torch.load('/home/jok120/ml/proj/data/old_data/tags/tmp/abrowntable.tch')

In [15]:
len(m), len(d)

(1644, 1645)

In [ ]:
!ls /home/jok120/ml/proj/data/old_data/tags/tmp/*.pkl

In [23]:
import os
import torch
import pickle

dfolder = '/home/jok120/ml/proj/data/old_data/tags/tmp/'
wfolder = '/home/jok120/ml/training_data/small'


X_train = torch.empty((0,3,xdim,ydim))
X_meta = []

N = 1

for f in os.listdir(dfolder):
    fname, fext = os.path.splitext(f)
    
    if fext != '.tch':
        continue
        
    X_train = torch.cat([X_train,torch.load(os.path.join(dfolder,f))])
    X_meta += pickle.load(open(os.path.join(dfolder,fname+'_metadata.pkl'),'rb'))
    print(len(X_train),len(X_meta))
    if len(X_train) > 20000:
        torch.save(X_train,os.path.join(wfolder,'train_data_{}.tch'.format(N)))
        pickle.dump(X_meta,open(os.path.join(wfolder,'train_meta_{}.pkl'.format(N)),'wb'))
        
        N += 1
        
        X_train = torch.empty((0,3,xdim,ydim))
        X_meta = []
        
torch.save(X_train,os.path.join(wfolder,'train_data_{}.tch'.format(N)))
pickle.dump(X_meta,open(os.path.join(wfolder,'train_meta_{}.pkl'.format(N)),'wb'))

0 0
18 18
1870 1870
1882 1882
2045 2045
2445 2445
3045 3045
3851 3851
5549 5549
6569 6569
7514 7514
9814 9814
10054 10054
13557 13557
14426 14426
14550 14550
14550 14550
16136 16136
19024 19024
19024 19024
19450 19450
22212 22212
770 770
1461 1461
1830 1830
2294 2294
6406 6406
6727 6727
8702 8702
9625 9625
10353 10353
13160 13160
13477 13477
13477 13477
15530 15530
18067 18067
25994 25994
8763 8763
11129 11129
13588 13588
14627 14627
23648 23648
0 0
426 426
693 693
819 819
1388 1388
2796 2796
6950 6950
7093 7093
8476 8476
8741 8741
9314 9314
11391 11391
12032 12032
12048 12048
12924 12924
13363 13363
15882 15882
19273 19273
20106 20106
1468 1468
11425 11425
12841 12841
14485 14485
19977 19977
20295 20295
3 3
1561 1561
7016 7016
10251 10251
11435 11435
11753 11753
14356 14356
17637 17637
18108 18108
19158 19158
19294 19294
21361 21361
3469 3469
5251 5251
5916 5916
6171 6171
7002 7002
8007 8007
8590 8590
9103 9103
10452 10452
11770 11770
15056 15056
15744 15744
17506 17506
19613 19613
19

In [1]:
import torch
a = torch.empty((0,3,64,64))
torch.cat([torch.ones(0,3,64,64),torch.ones(8,3,64,64)]).shape

torch.Size([8, 3, 64, 64])

In [24]:
torch.save(images,os.path.join(dfolder,'data/#homecooking.tch'.format(folder)))
pkl.dump(metadata,open(os.path.join(dfolder,'data/{}_metadata.pkl'.format(folder)),'wb'))